In [20]:
import pandas as pd
import numpy as np
from helper import zip_gz, unzip_gz
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb

In [21]:
df = unzip_gz("../data_per_crop/apples_df.gz")
df.head()

,target,year,Q1_cloud_cover.afternoon,Q2_cloud_cover.afternoon,Q3_cloud_cover.afternoon,Q4_cloud_cover.afternoon,Q1_humidity.afternoon,Q2_humidity.afternoon,Q3_humidity.afternoon,Q4_humidity.afternoon,...,Q4_temperature.night,Q1_wind.max.direction,Q2_wind.max.direction,Q3_wind.max.direction,Q4_wind.max.direction,Q1_wind.max.speed,Q2_wind.max.speed,Q3_wind.max.speed,Q4_wind.max.speed,is_future
0,100.000000,2022,17.200000,20.000000,42.500000,36.666667,51.400000,55.857143,59.333333,67.666667,...,279.750000,284.000000,165.714286,211.666667,140.000000,3.602000,8.452857,6.515000,7.716667,0
1,0.000000,2002,81.234000,41.437143,29.890000,84.173333,61.408000,51.948571,59.305000,74.846667,...,281.583333,152.252000,192.067143,205.256667,214.288333,5.896000,4.861429,5.345000,4.980000,0
2,90.781047,2022,28.200000,46.142857,20.000000,73.333333,59.400000,57.000000,59.166667,72.500000,...,281.871667,159.200000,216.714286,237.500000,160.333333,3.458000,5.607143,3.885000,4.723333,0
3,53.886067,2020,37.833333,53.500000,54.166667,71.833333,45.833333,62.166667,64.000000,82.000000,...,279.556667,198.333333,230.166667,204.666667,218.000000,6.516667,5.406667,3.323333,4.265000,0
4,77.647353,2020,37.833333,53.500000,54.166667,71.833333,45.833333,62.166667,64.000000,82.000000,...,279.556667,198.333333,230.166667,204.666667,218.000000,6.516667,5.406667,3.323333,4.265000,0


In [22]:
df.drop(columns=['is_future'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24182 entries, 0 to 24181
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   target                    24182 non-null  float64
 1   year                      24182 non-null  int64  
 2   Q1_cloud_cover.afternoon  24182 non-null  float64
 3   Q2_cloud_cover.afternoon  24182 non-null  float64
 4   Q3_cloud_cover.afternoon  24182 non-null  float64
 5   Q4_cloud_cover.afternoon  24182 non-null  float64
 6   Q1_humidity.afternoon     24182 non-null  float64
 7   Q2_humidity.afternoon     24182 non-null  float64
 8   Q3_humidity.afternoon     24182 non-null  float64
 9   Q4_humidity.afternoon     24182 non-null  float64
 10  Q1_precipitation.total    24182 non-null  float64
 11  Q2_precipitation.total    24182 non-null  float64
 12  Q3_precipitation.total    24182 non-null  float64
 13  Q4_precipitation.total    24182 non-null  float64
 14  Q1_pre

In [26]:
X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Train model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print(f"R-squared score: {r2:.4f}")

R-squared score: 0.2164


In [16]:
def within_threshold(actual, predicted, threshold):
    """
    Custom function to evaluate if the prediction is within a given threshold of the actual value.
    """
    return np.abs(actual - predicted) <= threshold

# Define your threshold (for example, 5 units)
threshold = 10

# Apply the threshold check
correct_predictions = within_threshold(y_test, y_pred, threshold)

# Calculate accuracy based on threshold
accuracy = np.mean(correct_predictions) * 100
print(f'Accuracy within threshold of {threshold}: {accuracy:.2f}%')

Accuracy within threshold of 10: 27.97%
